In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (10, 7)

In [2]:
data = pd.read_csv('/home/jvdzwaan/Downloads/1999_CHES_dataset_means.csv')
print data.shape
data[data['country']==10]

(142, 50)


,eastwest,eumember,country,year,expert,party_id,party,vote,family,govt,...,fin,fr,ger,gr,irl,it,nl,por,sw,uk
96,1,1,10,1999,11,1001,CDA,18.400000,4,0,...,0,0,0,0,0,0,1,0,0,0
97,1,1,10,1999,11,1002,PvdA,29.000000,5,1,...,0,0,0,0,0,0,1,0,0,0
98,1,1,10,1999,11,1003,VVD,24.699999,3,1,...,0,0,0,0,0,0,1,0,0,0
99,1,1,10,1999,11,1004,D'66,9.000000,3,1,...,0,0,0,0,0,0,1,0,0,0
100,1,1,10,1999,11,1005,GL,7.300000,7,0,...,0,0,0,0,0,0,1,0,0,0
101,1,1,10,1999,11,1006,SGP,1.800000,10,0,...,0,0,0,0,0,0,1,0,0,0
102,1,1,10,1999,11,1007,GPV,1.300000,10,0,...,0,0,0,0,0,0,1,0,0,0
103,1,1,10,1999,11,1008,RPF,2.000000,10,0,...,0,0,0,0,0,0,1,0,0,0
104,1,1,10,1999,11,1009,CD,0.000000,1,0,...,0,0,0,0,0,0,1,0,0,0
105,1,1,10,1999,11,1014,SP,3.500000,6,0,...,0,0,0,0,0,0,1,0,0,0


## Relevant variables

* FAMILY
* LRGEN (position of the party in YEAR in terms of its overall ideological stance)
* LRECON (osition of the party in YEAR in terms of its ideological stance on economic issues)
* GALTAN (position of the party in YEAR in terms of of their views on democratic freedoms and rights)
* SPENDVTAX (position on improving public services vs. reducing taxes)
* DEREGULATION (position on deregulation)
* REDISTRIBUTION (position on redistribution of wealth from the rich to the poor)
* ECON_INTERVEN (position on state intervention in the economy. (Only asked in 2014))
* CIVLIB_LAWORDER (position on civil liberties vs. law and order)
* SOCIALLIFESTYLE (position on social lifestyle (e.g. homosexuality))
* RELIGIOUS_PRINCIPLE (position on role of religious principles in politics)
* IMMIGRATE_POLICY (position on immigration policy)
* MULTICULTURALISM (position on integration of immigrants and asylum seekers (multiculturalism vs. assimilation))
* URBAN_RURAL (position on urban vs. rural interests)
* ENVIRONMENT (position towards the environment. (Asked in 2010 and 2014)
* REGIONS (position on political decentralization to regions/localities)
* INTERNATIONAL_SECURITY (position towards international security and peacekeeping missions. (Asked in 2010 and 2014)
* ETHNIC_MINORITIES (position towards ethnic minorities)

See http://chesdata.eu/1999-2014/1999-2014_CHES_codebook.pdf for full list (and explanations)

What variables to use?

* Only use ones which were asked in all years

How to deal with years vs. cabinets?

* Use average per cabinet?
* Start with one year

## Multiple Regression

Technique used for predicting the unknown value of a variable from the known value of two or more variables- also called the predictors.

The appropriateness of the multiple regression model as a whole can be tested by the F-test in the ANOVA table. A significant F indicates a linear relationship between Y and at least one of the X's.

https://explorable.com/multiple-regression-analysis


## Input Data

* nTopics (all high quality topics)
* opinions

NxV opinion matrix (cabinet-party) averaged?


In [4]:
from cptm.utils.experiment import load_opinions

config = {
    "inputData": "/path/to/input/data/*",
    "outDir": "/home/jvdzwaan/data/dilipad/results/all_years-adj-cabinets_selected-parties/{}",
    "nTopics": 100
}

opinions = load_opinions(config)

In [5]:
# select PvdA in Kok II
import numpy as np

perspectives = ['Kok II-PvdA', 'Kok II-SGP', 'Kok II-D66', 'Kok II-GroenLinks', 'Kok II-VVD', 'Kok II-SP']
print perspectives
data_in = np.zeros((opinions['Kok II-PvdA'].values.shape[0], len(perspectives)))
for i, p in enumerate(perspectives):
    # TODO: filter opinions (opinions on high quality topics)
    data_in[:, i] = opinions[p].values.sum(axis=1)
print data_in.shape

parties = perspectives = ['PvdA', 'SGP', 'D66', 'GroenLinks', 'VVD', 'SP']
data_out = np.array([data.loc[97, 'lrgen'], data.loc[101, 'lrgen'], data.loc[99, 'lrgen'], data.loc[100, 'lrgen'], data.loc[98, 'lrgen'], data.loc[105, 'lrgen']])
print data_out

['Kok II-PvdA', 'Kok II-SGP', 'Kok II-D66', 'Kok II-GroenLinks', 'Kok II-VVD', 'Kok II-SP']
(6245, 6)
[ 3.79999995  7.69999981  4.4000001   2.5999999   6.9000001   1.10000002]


In [7]:
import statsmodels.api as sm

data_in = sm.add_constant(data_in.T)
est = sm.OLS(data_out, data_in).fit()

est.summary()

/home/jvdzwaan/.virtualenvs/dilip/local/lib/python2.7/site-packages/statsmodels/stats/stattools.py:72: UserWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  "samples were given." % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Thu, 10 Dec 2015   Prob (F-statistic):                nan
Time:                        16:49:59   Log-Likelihood:                 162.70
No. Observations:                   6   AIC:                            -313.4
Df Residuals:                       0   BIC:                            -314.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.9359        inf          0        nan           nan       nan
x1             0.0078        inf          0        nan           nan       nan
x2             0.0536        inf          0        nan           nan       nan
x3             0.0704        inf          0        nan           nan       nan
x4             0.0198        inf          0        nan           nan       nan
x5            -0.1247        inf         -0        nan           nan       nan
x6             0.0061        inf          0        nan           nan       nan
x7             0.0114        inf          0        nan           nan       nan
x8             0.0167        inf          0        nan           nan       nan
x9             0.0087        inf          0        nan           nan       nan
x10           -0.0534        inf         -0        nan           nan       nan
x11            0.0065        inf          0        nan           nan       nan
x12            0.0563        inf          0        nan           nan       nan
x13            0.0745        inf          0        nan           nan       nan
x14            0.0475        inf          0        nan           nan       nan
x15            0.0195        inf          0        nan           nan       nan
x16            0.0034        inf          0        nan           nan       nan
x17            0.0114        inf          0        nan           nan       nan
x18            0.0120        inf          0        nan           nan       nan
x19            0.0010        inf          0        nan           nan       nan
x20            0.0269        inf          0        nan           nan       nan
x21            0.0114        inf          0        nan           nan       nan
x22            0.0198        inf          0        nan           nan       nan
x23            0.0033        inf          0        nan           nan       nan
x24            0.0626        inf          0        nan           nan       nan
x25            0.0008        inf          0        nan           nan       nan
x26            0.0084        inf          0        nan           nan       nan
x27            0.0114        inf          0        nan           nan       nan
x28           -0.0059        inf         -0        nan           nan       nan
x29            0.0166        inf          0        nan           nan       nan
x30            0.0330        inf          0        nan           nan       nan
x31           -0.3756        inf         -0        nan           nan       nan
x32            0.0114        inf          0        nan           nan       nan
x33            0.0150        inf          0        nan           nan       nan
x34           -0.0660        inf         -0        nan           nan       nan
x35            0.0114        inf          0        nan           nan       nan
x3

In [ ]:
# The coefficients
print('Coefficients: \n', clf.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((clf.predict(data_in.T) - data_out) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % clf.score(data_in.T, data_out))